In [1]:
from imp import reload
import LinearImageClassifier.FeatureMap as FeatureMap
reload(FeatureMap)
FeatureMap = FeatureMap.FeatureMap

import tensorflow as tf
import rasterio as rio
import geopandas as gpd
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as feature
import numpy as np
from skimage import img_as_ubyte
import matplotlib
import os
from rasterio.plot import show, show_hist

/home/ec2-user/miniconda3/envs/imageprocess/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
DATA_ROOT = "../data/"
imageindex_file = "train_v2.csv"
imagedir = 'train-tif-v2'
jpgdir = 'train-jpg'
img_template = "train_%d.%s"
imageindex = pd.read_csv(os.path.join(DATA_ROOT, imageindex_file))
labelTypes = set(" ".join(imageindex.tags.values).split(" "))
labelIDs = {l:i for l, i in zip(labelTypes, range(len(labelTypes)))}
print(labelIDs)

def _readimg(filename):
    return rio.open(filename).read()

def _onehot(labels, labelIDs):
    encoding = np.zeros(len(labelIDs))
    for label in labels.split(" "):
        encoding[labelIDs[label]] = 1
    return(np.array(encoding))
        
def imageparser(imgID, _):
    # input from tf dataset with (trainids, onehotlabels).map
    file = os.path.join(DATA_ROOT, imagedir, img_template %(imgID, 'tif'))
    data = rio.open(file).read()
    return(img_as_ubyte(data), _)
imageindex['onehot_tags'] = list(map(lambda row: _onehot(row[1].tags, labelIDs), imageindex.iterrows()))

{'selective_logging': 0, 'primary': 1, 'agriculture': 2, 'slash_burn': 3, 'cloudy': 4, 'bare_ground': 5, 'blooming': 6, 'partly_cloudy': 7, 'water': 8, 'artisinal_mine': 9, 'clear': 10, 'haze': 11, 'cultivation': 12, 'blow_down': 13, 'road': 14, 'habitation': 15, 'conventional_mine': 16}


In [3]:
trainids = imageindex.index.values
trainlabels = np.vstack(imageindex.onehot_tags.values)
dataset = tf.data.Dataset.from_tensor_slices((trainids[:1000], trainlabels[:1000]))

print(dataset)
dataset = dataset.map(
    lambda imgid, label: tuple(tf.py_func(
        imageparser, [imgid, label], [tf.uint8, tf.float64])))

iterator = tf.data.Iterator.from_structure(dataset.output_types,
                                           dataset.output_shapes)


<TensorSliceDataset shapes: ((), (17,)), types: (tf.int64, tf.float64)>


In [4]:
dataset

<MapDataset shapes: (<unknown>, <unknown>), types: (tf.uint8, tf.float64)>

In [5]:
import LinearImageClassifier.FeatureMap as FeatureMap
reload(FeatureMap)
FeatureMap = FeatureMap.FeatureMap
fm = FeatureMap(dataset, K=350, num_patches=20, patch_size=16, whiten=False)

In [6]:
fm

In [7]:
fm.fit()

0it [00:00, ?it/s]/home/ec2-user/miniconda3/envs/imageprocess/lib/python3.6/site-packages/rasterio/__init__.py:193: UserWarning: Dataset has no geotransform set.  Default transform will be applied (Affine.identity())
  s.start()
/home/ec2-user/miniconda3/envs/imageprocess/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from uint16 to uint8
  .format(dtypeobj_in, dtypeobj_out))
1000it [00:06, 170.69it/s]


In [8]:
len(fm.estimator.cluster_centers_)

350

In [9]:
fm.estimator.cluster_centers_[0].shape

(1024,)

In [10]:
fm.predict(fm._getRandomPatches())

TypeError: _getRandomPatches() missing 1 required positional argument: 'image'

In [17]:
import LinearImageClassifier.FeatureExtractor as FeatureExtractor
reload(FeatureExtractor)

<module 'LinearImageClassifier.FeatureExtractor' from '/home/ec2-user/planet/code/LinearImageClassifier/FeatureExtractor.py'>

In [18]:
fe = FeatureExtractor.FeatureExtractor(fm, stride=5)

In [19]:
fe._generatePatches()

TypeError: _generatePatches() missing 1 required positional argument: 'image'

In [20]:

images = imageindex
randimg_id = np.random.choice(len(images))
randimg = rio.open(os.path.join(DATA_ROOT, imagedir, img_template % (randimg_id, 'tif')))
img = randimg.read()

/home/ec2-user/miniconda3/envs/imageprocess/lib/python3.6/site-packages/rasterio/__init__.py:193: UserWarning: Dataset has no geotransform set.  Default transform will be applied (Affine.identity())
  s.start()


In [21]:
patches = fe._generatePatches(img)
patches.shape

(169, 1024)

In [22]:
fe.extract(dataset)


0it [00:00, ?it/s]/home/ec2-user/miniconda3/envs/imageprocess/lib/python3.6/site-packages/rasterio/__init__.py:193: UserWarning: Dataset has no geotransform set.  Default transform will be applied (Affine.identity())
  s.start()
/home/ec2-user/miniconda3/envs/imageprocess/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from uint16 to uint8
  .format(dtypeobj_in, dtypeobj_out))

1it [00:00,  7.10it/s]
2it [00:00,  6.95it/s]
3it [00:00,  6.86it/s]
4it [00:00,  6.81it/s]
5it [00:00,  6.76it/s]
6it [00:00,  6.75it/s]
7it [00:01,  6.68it/s]
8it [00:01,  6.40it/s]
9it [00:01,  6.47it/s]
10it [00:01,  6.50it/s]
11it [00:01,  6.55it/s]
12it [00:01,  6.56it/s]
13it [00:01,  6.52it/s]
14it [00:02,  6.48it/s]
15it [00:02,  6.47it/s]
16it [00:02,  6.48it/s]
17it [00:02,  6.46it/s]
18it [00:02,  6.21it/s]
19it [00:02,  6.28it/s]
20it [00:03,  6.33it/s]
21it [00:03,  6.36it/s]
22it [00:03,  6.38it/s]
23it [00:03,  6.39it/s]
24it [00:03,  6

281it [00:45,  6.13it/s]
282it [00:45,  6.14it/s]
283it [00:45,  6.15it/s]
284it [00:45,  6.16it/s]
285it [00:45,  5.85it/s]
286it [00:45,  5.94it/s]
287it [00:46,  6.01it/s]
288it [00:46,  6.06it/s]
289it [00:46,  6.08it/s]
290it [00:46,  6.12it/s]
291it [00:46,  6.14it/s]
292it [00:46,  6.16it/s]
293it [00:47,  6.17it/s]
294it [00:47,  6.16it/s]
295it [00:47,  5.84it/s]
296it [00:47,  5.95it/s]
297it [00:47,  6.01it/s]
298it [00:47,  6.07it/s]
299it [00:48,  6.10it/s]
300it [00:48,  6.14it/s]
301it [00:48,  6.14it/s]
302it [00:48,  6.14it/s]
303it [00:48,  6.15it/s]
304it [00:48,  5.85it/s]
305it [00:49,  5.94it/s]
306it [00:49,  6.00it/s]
307it [00:49,  6.05it/s]
308it [00:49,  6.08it/s]
309it [00:49,  6.11it/s]
310it [00:49,  6.11it/s]
311it [00:49,  6.12it/s]
312it [00:50,  6.12it/s]
313it [00:50,  5.84it/s]
314it [00:50,  5.86it/s]
315it [00:50,  5.96it/s]
316it [00:50,  6.02it/s]
317it [00:50,  6.07it/s]
318it [00:51,  6.09it/s]
319it [00:51,  6.12it/s]
320it [00:51,  6.13it/s]


573it [01:34,  5.55it/s]
574it [01:34,  5.67it/s]
575it [01:34,  5.76it/s]
576it [01:34,  5.83it/s]
577it [01:34,  5.87it/s]
578it [01:34,  5.90it/s]
579it [01:35,  5.93it/s]
580it [01:35,  5.94it/s]
581it [01:35,  5.95it/s]
582it [01:35,  5.53it/s]
583it [01:35,  5.67it/s]
584it [01:35,  5.76it/s]
585it [01:36,  5.82it/s]
586it [01:36,  5.86it/s]
587it [01:36,  5.87it/s]
588it [01:36,  5.89it/s]
589it [01:36,  5.90it/s]
590it [01:36,  5.90it/s]
591it [01:37,  5.48it/s]
592it [01:37,  5.61it/s]
593it [01:37,  5.69it/s]
594it [01:37,  5.77it/s]
595it [01:37,  5.81it/s]
596it [01:38,  5.85it/s]
597it [01:38,  5.87it/s]
598it [01:38,  5.90it/s]
599it [01:38,  5.91it/s]
600it [01:38,  5.50it/s]
601it [01:38,  5.63it/s]
602it [01:39,  5.73it/s]
603it [01:39,  5.81it/s]
604it [01:39,  5.85it/s]
605it [01:39,  5.88it/s]
606it [01:39,  5.89it/s]
607it [01:39,  5.91it/s]
608it [01:40,  5.92it/s]
609it [01:40,  5.49it/s]
610it [01:40,  5.62it/s]
611it [01:40,  5.73it/s]
612it [01:40,  5.76it/s]


865it [02:25,  5.62it/s]
866it [02:25,  5.63it/s]
867it [02:26,  5.66it/s]
868it [02:26,  5.21it/s]
869it [02:26,  5.36it/s]
870it [02:26,  5.46it/s]
871it [02:26,  5.53it/s]

872it [02:27,  4.80it/s]
873it [02:27,  5.04it/s]
874it [02:27,  5.20it/s]
875it [02:27,  5.29it/s]
876it [02:27,  5.03it/s]
877it [02:28,  5.18it/s]
878it [02:28,  5.29it/s]
879it [02:28,  5.35it/s]
880it [02:28,  5.41it/s]
881it [02:28,  5.44it/s]
882it [02:28,  5.47it/s]
883it [02:29,  5.49it/s]
884it [02:29,  5.51it/s]
885it [02:29,  5.27it/s]
886it [02:29,  5.36it/s]
887it [02:29,  5.41it/s]
888it [02:30,  5.44it/s]
889it [02:30,  5.45it/s]
890it [02:30,  5.48it/s]
891it [02:30,  5.50it/s]
892it [02:30,  5.51it/s]
893it [02:30,  5.15it/s]
894it [02:31,  5.27it/s]
895it [02:31,  5.33it/s]
896it [02:31,  5.39it/s]
897it [02:31,  5.43it/s]
898it [02:31,  5.46it/s]
899it [02:32,  5.49it/s]
900it [02:32,  5.50it/s]
901it [02:32,  5.51it/s]
902it [02:32,  5.27it/s]
903it [02:32,  5.33it/s]
904it [02:32,  5.37it/s]

In [ ]:
?fm.estimator.predict